### IMPORTS

In [1]:
# MYSQL
import mysql.connector
from mysql.connector import Error

# AI
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# SCRAPPING
import urllib
from urllib import request
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

# OTHERS
import pandas as pd
import numpy as np
import time
import re

### DB Works

In [2]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def create_database_connection(host_name, user_name, user_password, database_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=database_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [3]:
connection = create_database_connection("localhost", "root", "angel11", "footballscience")

MySQL Database connection successful


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# C:\Projets\extra\chromedriver-win32
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("enable-automation")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-extensions")
options.add_argument("--dns-prefetch-disable")
options.add_argument("--disable-gpu")

service = Service('C:/Projets/extra/chromedriver-win32/chromedriver.exe')
service.start()

driver = webdriver.Chrome(service=service, options = options)

In [5]:
# POUR LE MOMENT BLOQUE : transfert_markt = "https://www.transfermarkt.fr/"
sofifa = "https://sofifa.com/players"
football_observatory = "https://football-observatory.com/IMG/sites/playerprofile/"
whoscored = "https://www.whoscored.com/"

### Scrapping sofifa Player Statistics

In [6]:
path = "https://sofifa.com/players?type=all&lg%5B0%5D=13&lg%5B1%5D=16&lg%5B2%5D=19&lg%5B3%5D=31&lg%5B4%5D=53&offset=120" #"https://sofifa.com/players"
driver.get(path)

In [7]:
players = driver.find_element('xpath','/html/body/main/article/table')
#print(players.text)

In [8]:
liste_test = re.split(" ~ | |\n", players.text)
liste_test = liste_test[12:]
#print(liste_test)

In [42]:
liste_test#[0:10]

['Gabriel',
 'Jesus',
 'ST',
 'RW',
 '26',
 '83',
 '84',
 'Arsenal',
 '2022',
 '2027',
 '€43.5M',
 '€150K',
 '2109',
 'Pau',
 'Torres',
 'CB',
 '26',
 '83',
 '86',
 'Aston',
 'Villa',
 '2023',
 '2028',
 '€42M',
 '€100K',
 '1915',
 'R.',
 'Sterling',
 'LM',
 'RM',
 'ST',
 '28',
 '83',
 '83',
 'Chelsea',
 '2022',
 '2027',
 '€36.5M',
 '€145K',
 '2080',
 'R.',
 'James',
 'RB',
 'RWB',
 '23',
 '83',
 '85',
 'Chelsea',
 '2017',
 '2028',
 '€43.5M',
 '€105K',
 '2255',
 'Thiago',
 'CM',
 '32',
 '83',
 '83',
 'Liverpool',
 '2020',
 '2024',
 '€28.5M',
 '€135K',
 '2154',
 'C.',
 'Gakpo',
 'CF',
 'LW',
 '24',
 '83',
 '87',
 'Liverpool',
 '2023',
 '2028',
 '€52.5M',
 '€130K',
 '2073',
 'A.',
 'Onana',
 'GK',
 '27',
 '83',
 '84',
 'Manchester',
 'United',
 '2023',
 '2028',
 '€31M',
 '€110K',
 '1496',
 'M.',
 'Rashford',
 'LW',
 'RW',
 'ST',
 '25',
 '83',
 '84',
 'Manchester',
 'United',
 '2015',
 '2028',
 '€43.5M',
 '€155K',
 '2034',
 'S.',
 'Botman',
 'CB',
 '23',
 '83',
 '87',
 'Newcastle',
 'Unite

In [56]:
names = []
surnames = []
positions = []
ages = []
grades = []
potentials = []
values = []
wages = []
contract_starts = []
contract_ends = []
clubs = []

positions_sofifa = ['GK','RB','RWB','LB','LWB','CB','CDM','CM','CAM','LM','LW','RM','RW','ST','LF','RF','CF']

i = 0
while i < len(liste_test):
    grpt = 0
    contract_end = False
    position = ""
    club = ""
    names.append(liste_test[i])
    i = i+1
    if liste_test[i] not in positions_sofifa:
        surnames.append(liste_test[i])
        i = i+1
    else:
        surnames.append("")

    while  "€" not in liste_test[i]:
        try:
            val = int(liste_test[i])
            if val < 100 and grpt == 0:
                ages.append(liste_test[i])
                grpt = grpt+1
                i = i+1
            elif val < 100 and grpt == 1:
                grades.append(liste_test[i])
                grpt = grpt+1
                i = i+1
            elif val < 100 and grpt == 2:
                potentials.append(liste_test[i])
                i = i+1
            elif val > 1950 and contract_end == False:
                contract_starts.append(liste_test[i])
                contract_end = True
                i = i+1
            elif val > 1950 and contract_end == True:
                contract_ends.append(liste_test[i])
                i = i+1
        except:
            if liste_test[i] not in positions_sofifa:
                if len(position)>0:
                    club = club + " " + liste_test[i]
                else:
                    club = liste_test[i]
                i = i+1
            else:
                if len(position)>0:
                    position = position + " " + liste_test[i]
                else:
                    position = liste_test[i]
                i = i+1
    clubs.append(club)
    positions.append(position)
    try:
        value = liste_test[i].replace("M","")
        value = value.replace("€","")
        value = value.replace("K","000")
        wage = liste_test[i+1].replace("M","")     #travail a faire sur les valuers ==> .5 => on enleve un 0 à ajouter
        wage = wage.replace("€","")
        wage = wage.replace("K","")
        value = float(value)
        wage = float(wage)
        values.append(liste_test[i]) # value)
        wages.append(wage)
    except:
        values.append("")
        wages.append("")
    i = i+3
    #print("at the end of the while")
            

In [51]:
#print(ages)

In [59]:
df = pd.DataFrame( {'name':names,'surname':surnames,'position':positions,
                    'age':ages,'grade':grades,'potential':potentials,'value':values,
                    'wage':wages,'contract_start':contract_starts,'contract_end':contract_ends,'club':clubs
                    })

In [60]:
df

,name,surname,position,age,grade,potential,value,wage,contract_start,contract_end,club
0,Gabriel,Jesus,ST RW,26,83,84,43.5,150.0,2022,2027,Arsenal
1,Pau,Torres,CB,26,83,86,42.0,100.0,2023,2028,Aston Villa
2,R.,Sterling,LM RM ST,28,83,83,36.5,145.0,2022,2027,Chelsea
3,R.,James,RB RWB,23,83,85,43.5,105.0,2017,2028,Chelsea
4,Thiago,,CM,32,83,83,28.5,135.0,2020,2024,Liverpool
5,C.,Gakpo,CF LW,24,83,87,52.5,130.0,2023,2028,Liverpool
6,A.,Onana,GK,27,83,84,31.0,110.0,2023,2028,Manchester United
7,M.,Rashford,LW RW ST,25,83,84,43.5,155.0,2015,2028,Manchester United
8,S.,Botman,CB,23,83,87,47.0,105.0,2022,2027,Newcastle United
9,C.,Romero,CB,25,83,87,45.5,95.0,2022,2027,Tottenham Hotspur


In [33]:
zip(names,surnames,positions,ages,grades,potentials,values,wages,contract_starts,contract_ends,clubs)

### Annexes 

In [6]:
driver.get(football_observatory)
# liste = driver.find_element('xpath','/html/body/div/div/div/div[2]/div[2]')
# print(liste.text)

In [72]:
search_box = driver.find_element('xpath','//*[@id="player"]')
search_box.clear()
search_box.send_keys("Rúben Dias")
auto_complete = driver.find_elements('xpath', '//*[@id="player"]')
auto_complete[0].click()

In [73]:
liste2 = driver.find_element('xpath','/html/body/div/div/div/div[2]/div[2]')

In [74]:
liste2.text

'Performance per game area\nRatio to team (×)\nRatio to league (×)\n1\nAir defence\n3.10\n3.10\n2\nGround defence\n1.87\n1.93\n3\nAir attack\n1.74\n1.69\n4\nDistribution\n1.34\n1.78\n5\nRecovery\n1.28\n1.43\n6\nShooting\n0.49\n0.75\n7\nChance creation\n0.12\n0.25\n8\nTake on\n0.06\n0.07'

In [ ]:
search_box = driver.find_element('xpath','//*[@id="player"]')
search_box.clear()
search_box.send_keys("Hugo")
auto_complete = driver.find_elements('xpath', '/html/body/div[2]/ul/li[1]')
auto_complete[0].click()

In [75]:
liste3 = driver.find_element('xpath','/html/body/div/div/div/div[2]/div[2]')

In [76]:
test = liste3.text.split("\n")
test

['Performance per game area',
 'Ratio to team (×)',
 'Ratio to league (×)',
 '1',
 'Air defence',
 '3.10',
 '3.10',
 '2',
 'Ground defence',
 '1.87',
 '1.93',
 '3',
 'Air attack',
 '1.74',
 '1.69',
 '4',
 'Distribution',
 '1.34',
 '1.78',
 '5',
 'Recovery',
 '1.28',
 '1.43',
 '6',
 'Shooting',
 '0.49',
 '0.75',
 '7',
 'Chance creation',
 '0.12',
 '0.25',
 '8',
 'Take on',
 '0.06',
 '0.07']

In [ ]:
#script pour récuperer les statistiques des joueurs
#listes noms / listes air defence ...
stats = []
liste_joueur_non_present = []
search_box = driver.find_element('xpath','//*[@id="player"]')
for player in liste_joueurs:
    search_box.clear()
    search_box.send_keys(player)
    time.sleep(1)
    auto_complete = driver.find_elements('xpath', '/html/body/div[2]/ul/li[1]')
    try:
        auto_complete[0].click()
        liste3 = driver.find_element('xpath','/html/body/div/div/div/div[2]/div[2]')
        liste3 = liste3.text.split("\n")
        stat = [liste3[5],liste3[6],liste3[9],liste3[10],liste3[13],liste3[14],liste3[17],liste3[18],liste3[21],liste3[22],liste3[25],liste3[26],liste3[29],liste3[30],liste3[33],liste3[34]]
        stats.append(stat)
    except:
        liste_joueur_non_present.append(player)

In [14]:
connection.close()

In [53]:
print(connection.database)

footballscience
